In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


df = pd.read_csv('trading_data.csv', index_col='timestamp')
total_rows = len(df)
print(total_rows)
df.head()

132194


,bid_price,bid_qty,ask_price,ask_qty,trade_price,sum_trade_1s,bid_advance_time,ask_advance_time,last_trade_time,_1s_side,_3s_side,_5s_side
timestamp,,,,,,,,,,,,
2018-01-31 23:45:11.445630+00:00,23270.0,110.0,23280.0,68.0,23270.0,106.0,9.491206,7.492440,0.000078,1,1,1
2018-01-31 23:45:11.446141+00:00,23270.0,10.0,23280.0,68.0,23270.0,107.0,9.491717,7.492951,0.000458,1,1,1
2018-01-31 23:45:11.446206+00:00,23270.0,8.0,23280.0,68.0,23270.0,107.0,9.491782,7.493016,0.000523,1,1,1
2018-01-31 23:45:11.446253+00:00,23270.0,3.0,23280.0,70.0,23270.0,107.0,9.491829,7.493063,0.000570,1,1,1
2018-01-31 23:45:11.446279+00:00,23270.0,1.0,23280.0,70.0,23270.0,107.0,9.491855,7.493089,0.000596,0,0,0


In [2]:
df.isnull().sum(axis=0)

bid_price               0
bid_qty                 0
ask_price               0
ask_qty                 0
trade_price             0
sum_trade_1s        65975
bid_advance_time        0
ask_advance_time        0
last_trade_time      9591
_1s_side                0
_3s_side                0
_5s_side                0
dtype: int64

In [ ]:
import seaborn as sns
sns.pairplot(df)

C:\Users\TP_baseline\Anaconda3\lib\site-packages\numpy\lib\histograms.py:754: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
C:\Users\TP_baseline\Anaconda3\lib\site-packages\numpy\lib\histograms.py:755: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


In [10]:
# set different Ys for prediction, drop ask and trade due to correlation
df_1s = df.drop(['_3s_side','_5s_side','ask_price','trade_price'], axis = 1)
df_1s_x = df_1s.drop(["_1s_side"], axis = 1)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_1s_x)
scaled_1s = pd.DataFrame(scaled_data, index=df_1s_x.index, columns=df_1s_x.columns)
scaled_1s['_1s_side'] = df_1s['_1s_side']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

#imput missing values in sum_trade_1s and last_trade_time

fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
imputed_df_1s = pd.DataFrame(fill_NaN.fit_transform(scaled_1s))
imputed_df_1s.columns = scaled_1s.columns
imputed_df_1s.index = scaled_1s.index

X = imputed_df_1s.drop(['_1s_side'], axis =1)
y = imputed_df_1s['_1s_side']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle = False)

C:\Users\TP_baseline\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [17]:
from imblearn.over_sampling import SMOTENC
sm = SMOTENC(random_state=42, categorical_features=[6])
X_resampled, y_resampled = sm.fit_sample(X_train, y_train)

In [18]:
print("Before OverSampling, counts of label '0': {}".format(sum(pd.DataFrame(y_train)['_1s_side']==0)))
print("Before OverSampling, counts of label '1': {}".format(sum(pd.DataFrame(y_train)['_1s_side']==1)))
print("Before OverSampling, counts of label '2': {}".format(sum(pd.DataFrame(y_train)['_1s_side']==2)))
print("After OverSampling, counts of label '0': {}".format(sum(pd.DataFrame(y_resampled)[0]==0)))
print("After OverSampling, counts of label '1': {}".format(sum(pd.DataFrame(y_resampled)[0]==1)))
print("After OverSampling, counts of label '2': {}".format(sum(pd.DataFrame(y_resampled)[0]==2)))


Before OverSampling, counts of label '0': 71341
Before OverSampling, counts of label '1': 10085
Before OverSampling, counts of label '2': 11109
After OverSampling, counts of label '0': 71341
After OverSampling, counts of label '1': 71341
After OverSampling, counts of label '2': 71341


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

knn = KNeighborsClassifier(n_neighbors = 461) #square root of n
knn.fit(X_resampled, y_resampled)

# Accuracy of k-NN model
y_pred = knn.predict(X_test)
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
print('Accuracy: {:.5f}'.format(knn.score(X_test, y_test)))

print(classification_report(y_test, knn.predict(X_test)))

Misclassified samples: 15508
Accuracy: 0.60897
              precision    recall  f1-score   support

         0.0       0.93      0.55      0.69     30870
         1.0       0.35      0.82      0.49      4324
         2.0       0.33      0.82      0.47      4465

   micro avg       0.61      0.61      0.61     39659
   macro avg       0.54      0.73      0.55     39659
weighted avg       0.80      0.61      0.64     39659



In [20]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 10 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_resampled, y_resampled);


In [28]:

# Use the forest's predict method on the test data
rf_pred = rf.predict(X_test)
count_misclassified = (y_test != rf_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
print('Accuracy: {:.5f}'.format(rf.score(X_test, y_test)))
print(classification_report(y_test, rf.predict(X_test)))

Misclassified samples: 8321
Accuracy: 0.79019
              precision    recall  f1-score   support

         0.0       0.88      0.85      0.87     30870
         1.0       0.52      0.51      0.52      4324
         2.0       0.51      0.64      0.56      4465

   micro avg       0.79      0.79      0.79     39659
   macro avg       0.64      0.67      0.65     39659
weighted avg       0.80      0.79      0.79     39659



In [22]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 6).fit(X_resampled, y_resampled) 
dtree_predictions = dtree_model.predict(X_test)
d_tree_count_misclassified = (y_test != dtree_predictions).sum()
print('Misclassified samples: {}'.format(d_tree_count_misclassified))
print('Accuracy: {:.5f}'.format(dtree_model.score(X_test, y_test)))
print(classification_report(y_test, dtree_model.predict(X_test)))

Misclassified samples: 13871
Accuracy: 0.65024
              precision    recall  f1-score   support

         0.0       0.95      0.59      0.73     30870
         1.0       0.38      0.86      0.52      4324
         2.0       0.36      0.84      0.50      4465

   micro avg       0.65      0.65      0.65     39659
   macro avg       0.56      0.76      0.59     39659
weighted avg       0.82      0.65      0.68     39659



In [26]:
# training a linear SVM classifier 
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'rbf', C = 10, gamma = 1.0).fit(X_resampled, y_resampled) 

In [27]:
svm_predictions = svm_model_linear.predict(X_test)
svm_count_misclassified = (y_test != svm_predictions).sum()
print('Misclassified samples: {}'.format(svm_count_misclassified))
print('Accuracy: {:.5f}'.format(svm_model_linear.score(X_test, y_test)))
print(classification_report(y_test, svm_model_linear.predict(X_test)))

Misclassified samples: 10531
Accuracy: 0.73446
              precision    recall  f1-score   support

         0.0       0.89      0.76      0.82     30870
         1.0       0.45      0.69      0.55      4324
         2.0       0.40      0.62      0.49      4465

   micro avg       0.73      0.73      0.73     39659
   macro avg       0.58      0.69      0.62     39659
weighted avg       0.79      0.73      0.75     39659



In [35]:
from sklearn. ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
adb = AdaBoostClassifier(DecisionTreeClassifier(),n_estimators = 20, learning_rate = 1)
adb_model= adb.fit(X_resampled,y_resampled)

In [37]:
adb_pred = adb_model.predict(X_test)
adb_count_misclassified = (y_test != adb_pred).sum()
print('Misclassified samples: {}'.format(adb_count_misclassified))
print('Accuracy: {:.5f}'.format(adb_model.score(X_test, y_test)))
print(classification_report(y_test, adb_model.predict(X_test)))

Misclassified samples: 11679
Accuracy: 0.70551
              precision    recall  f1-score   support

         0.0       0.86      0.77      0.81     30870
         1.0       0.35      0.43      0.39      4324
         2.0       0.36      0.54      0.43      4465

   micro avg       0.71      0.71      0.71     39659
   macro avg       0.52      0.58      0.54     39659
weighted avg       0.75      0.71      0.72     39659



In [41]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
dt = DecisionTreeClassifier()
svm = SVC(kernel = 'rbf', C = 10, gamma = 1.0 )
evc = VotingClassifier( estimators= [('lr',lr),('dt',dt),('svm',svm)], voting = 'hard')

In [42]:
evc.fit(X_resampled,y_resampled)

C:\Users\TP_baseline\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\TP_baseline\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)), ('dt', Decision...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [43]:
evc.score(X_test, y_test)

0.7284349075871808